In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import torchvision
from torchvision import datasets, transforms
import torch.utils
import torch.utils.data 
import torch.utils.data.dataloader
import torchvision.models.resnet

from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.metrics import f1_score
from itertools import groupby

In [3]:
print(f"PyTorch version: {torch.__version__}")

PyTorch version: 2.4.0


In [4]:
print(f"Is MPS built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set device
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Is MPS built? True
Is MPS available? True
Using device: mps


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

# 2. Laden des AlexNet Modells ohne vortrainierte Gewichte
#alexnet = models.alexnet(pretrained=True)
alexnet = models.resnet50(pretrained=True)
#alexnet = models.vgg16(pretrained=True)
alexnet = alexnet.to(device)

# 3. & 4. Datensatz vorbereiten und in Trainings- und Validierungssets aufteilen
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 5. Laden des Datensatzes

# Absoluter Pfad basierend auf dem aktuellen Arbeitsverzeichnis
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Relative path to the folder containing the images
relative_folderpath = os.path.join(parent_directory, 'data/segmented/graph/pl')

# Erstellen des Datasets
dataset = datasets.ImageFolder(relative_folderpath, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

random_seed = 2024
torch.manual_seed(random_seed)

# Define the sizes of the train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Split the dataset into train and test sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
print('Train Dataset: ' + str(len(train_dataset)))
print('Test Dataset: ' + str(len(test_dataset)))


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=4)

# 6. Verlustfunktion und Optimierer definieren
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

# 7. Trainingsschleife (vereinfacht)
for epoch in range(10):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch: {epoch + 1} | Loss: {loss.item()}')

    # Hier könnte eine Validierungsschleife folgen

/Users/davidpichler/GitHubRepo/Thesis/.conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/davidpichler/GitHubRepo/Thesis/.conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Train Dataset: 578
Test Dataset: 145
Epoch: 1 | Loss: 1.1803280115127563
Epoch: 2 | Loss: 1.9388232231140137
Epoch: 3 | Loss: 0.6202244758605957
Epoch: 4 | Loss: 0.18529725074768066
Epoch: 5 | Loss: 2.0576059818267822
Epoch: 6 | Loss: 0.8225549459457397
Epoch: 7 | Loss: 0.05219484865665436
Epoch: 8 | Loss: 0.03439133241772652
Epoch: 9 | Loss: 3.2476890087127686
Epoch: 10 | Loss: 0.6995067000389099


In [6]:
alexnet.eval()

with torch.no_grad():
   correct = 0
   total = 0
   true_labels = []
   predicted_labels = []

   for X_test_tensor, Y_test_tensor in test_loader:
      X_test_tensor, Y_test_tensor = X_test_tensor.to(device), Y_test_tensor.to(device)
      outputs = alexnet(X_test_tensor)
      _, predicted = torch.max(outputs.data, 1)

      y_test = Y_test_tensor.cpu().numpy()
      predicted = predicted.cpu().numpy()

      correct += (predicted == y_test).sum().item()
      total += len(y_test)


      true_labels.extend(y_test)
      predicted_labels.extend(predicted)

   unique, counts = np.unique(predicted_labels, return_counts=True)

   print(f'Predicted labels: {dict(zip(unique, counts))}')

   unique, counts = np.unique(true_labels, return_counts=True)
   print(f'True labels: {dict(zip(unique, counts))}')

   accuracy = correct / total

   print(f'Accuracy: {accuracy}')


Predicted labels: {np.int64(0): np.int64(68), np.int64(1): np.int64(74), np.int64(2): np.int64(3)}
True labels: {np.int64(0): np.int64(40), np.int64(1): np.int64(53), np.int64(2): np.int64(52)}
Accuracy: 0.5724137931034483


In [7]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# Create a DataFrame from the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
cm_df = pd.DataFrame(cm, index=['Actual Corrected', 'Actual Original', 'Actual Smoothed'], columns=['Predicted Corrected', 'Predicted Original', 'Predicted Smoothed'])

# Display the DataFrame
print(cm_df)


                  Predicted Corrected  Predicted Original  Predicted Smoothed
Actual Corrected                   37                   3                   0
Actual Original                     6                  45                   2
Actual Smoothed                    25                  26                   1


In [9]:
if not os.path.exists('model/'): os.makedirs('model/')
torch.save({
    'model_state_dict': alexnet.state_dict(),
    'model_architecture': alexnet
}, 'model/resNet_complete.pth')

In [ ]:
'''
 if average_validation_loss < best_validation_loss:
            best_validation_loss = average_validation_loss
            best_model_state_dict = model.state_dict()
            print(epoch, ' saved!')
'''